### Connect to Database and Load Data


In [ ]:
import sqlite3
import pandas as pd

In [ ]:
%load_ext sql
con = sqlite3.connect('./Database/spacex.db')
cur = con.cursor()

In [ ]:
%sql sqlite:///Database/spacex.db
df = pd.read_csv("./Datasets/spacex_IBM_data.csv")
df.to_sql('SPACEXTBL', con, if_exists='replace', index=False,method='multi')

In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

### SQL Queries


In [ ]:
# Display the names of the unique launch sites in the space mission
%sql SELECT DISTINCT launch_site AS "Launch Sites" FROM SPACEXTBL

In [ ]:
# Display 5 records where launch sites begin with the string CCA
%sql SELECT * FROM SPACEXTBL WHERE launch_site LIKE "CCA%" LIMIT 5

In [ ]:
# Display the total payload mass carried by boosters launched by NASA (CRS)
%sql SELECT SUM(payload_mass__kg_) AS "Total Payload Mass" FROM SPACEXTBL WHERE "Customer" = "NASA (CRS)"

In [ ]:
# Display average payload mass carried by booster version F9 v1.1
%sql SELECT AVG(payload_mass__kg_) AS "Average Payload Mass" FROM SPACEXTBL WHERE booster_version = "F9 v1.1"

In [ ]:
# List the date when the first succesful landing outcome in ground pad was acheived
%sql SELECT MIN(DATE) AS "First Successful Landing" FROM SPACEXTBL WHERE Landing_Outcome = "Success (ground pad)"

In [ ]:
# List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql SELECT booster_version FROM SPACEXTBL WHERE Landing_Outcome = "Success (drone ship)" AND payload_mass__kg_ BETWEEN 4000 AND 6000

In [ ]:
# List the total number of successful and failure mission outcomes
%sql SELECT COUNT(*) AS "Successful", (SELECT COUNT(*) FROM SPACEXTBL WHERE Mission_Outcome LIKE 'Failure%') AS "Failure" FROM SPACEXTBL WHERE Mission_Outcome LIKE "Success%"

In [ ]:
# List the names of the booster_versions which have carried the maximum payload mass
%sql SELECT booster_version, payload_mass__kg_ FROM SPACEXTBL WHERE payload_mass__kg_ = (SELECT MAX(payload_mass__kg_) FROM SPACEXTBL)

In [ ]:
# List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015
%sql SELECT Landing_Outcome, booster_version, launch_site FROM SPACEXTBL WHERE DATE LIKE "2015%" AND Landing_Outcome LIKE "Failure (drone ship)"

In [ ]:
# Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order
%sql SELECT Landing_Outcome, COUNT(*) AS "Count" FROM SPACEXTBL WHERE DATE BETWEEN "2010-06-04" AND "2017-03-20" GROUP BY Landing_Outcome ORDER BY "Count" DESC